# <center>第6章 数据加载、存储与文件格式</center>

## 6.1 读写文本格式的数据
>pandas提供了一些用于将表格型数据读取为DataFrame的函数,主要是read_csv

pd.read_csv(path,header=None,names,index_col)

In [1]:
import pandas as pd 
import numpy as np 

### 6.1.1 列名的指定(header和names参数)

In [6]:
#header参数不为None
path1=r'D:\文档\Python Scripts\data_test\ex1.csv'
df1=pd.read_csv(path1,header=0)
df2=pd.read_csv(path1,header=None)

#header参数为None,指定names参数
name=['Julia','Cindy','Mary','Job','Hack','Navi']
pd.read_csv(path1,names=name,index_col='Julia')

,Cindy,Mary,Job,Hack,Navi
Julia,,,,,
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo


### 6.1.2 指定某一列为行名

In [11]:
pd.read_csv(path1,names=name,index_col='Navi',skiprows=[0])
#如果希望将多个列做成一个层次化的索引，只需要传入由列编号组成的列标即可
pd.read_csv('D:\\文档\\Python Scripts\\data_test\\ex3.csv',header=0,index_col=['key1','key2'])
#你可以用skiprows跳过文件的第一行、第三行和第四行,如果指定了header，是不会被跳过的
#但是这个skiprows貌似是从1开始计数的，不懂
pd.read_csv(path1,skiprows=[1,3,4])

,Julia,Cindy,Mary,Job,Hack
Navi,,,,,
hello,1,1,2,3,4
world,2,5,6,7,8
foo,3,9,10,11,12


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

,0,a,b,c,d,message
0,2,5,6,7,8,world


read_csv的参数说明：

| 参数 | 说明 |
| :-: | :-: |
| path | 表示文件系统位置 |
| sep | 用于对行中各字段进行拆分的序列或者正则表达式 |
| header | 用作列名的行号，默认为0，如果没有，设置为None |
| index_col | 用作行索引的列编号或者列名，可以是单个名称或者是列表 |
| names | 结合header=None，传入列名 |
| skiprows | 需要忽略的行数 |
| na_values | 一组用于替换NA的值 |
| comment | 用于将注释信息从行尾拆分出去 |
| parse_dates | 尝试将数据解析为日期，默认为False，此外还可以指定需要解析的一组列号或者列名 |
| keep_data_col | 如果连接多列解析日期，则保持参与连接的列 |
| converters | 由列名跟函数之间的映射组成的字典，例如{'foo':f}会对foo列的所有值应用函数'f' |
| nrows | 需要读取的行数 |
| skip_footer | 需要忽略的函数（从文件末尾处算起） |

### 6.1.3 缺失值的处理(na_values参数)

In [ ]:
path=r"D:\文档\Python Scripts\data_test\ex1.csv"
d={'a':'na','message':'ss'}
pd.read_csv(path,na_values=d)

### 6.1.4 逐块读取文本文件

In [13]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import os
os.chdir(r'D:\文档\Python Scripts')

In [3]:
code='0700.hk'
start='2017-4-4'
end='2017-10-10'
info=web.get_data_yahoo(code,start,end)
info.to_csv('data_test\\stock_price.csv',header=1,index=True)

In [ ]:
pd.options.display.max_rows=10#设置pandas的显示

path=r'data_test\stock_price.csv'
#如果你只想读取几行，通过nrows指定
pd.read_csv(path,nrows=10)
#read_csv会自动将Date所在的行变为列名
df=pd.read_csv(path,index_col='Date')
#要逐块读取文件，可以指定chunksize(行数)
chunker=pd.read_csv(path,chunksize=100)
type(chunker)
#TextFileReader对象使你可以根据chunksize对文件进行迭代

### 6.1.5 将数据写出到文本

>DataFrame.to_csv(path_or_buf,na_rep='',columns=None,header=True,index=True,index_label=None,encoding=None)

In [ ]:
code='600519.ss'
start='2017-4-4'
end='2017-5-5'
info=web.get_data_yahoo(code,start,end)
info.to_csv('D:\\文档\\Python Scripts\\data_test\\600519ss_price.csv',index=False,na_rep='NULL')
#此外，你还可以只写出一部分的列，并以你指定的顺序排列
info.to_csv('D:\\文档\\Python Scripts\\data_test\\600519ss_price.csv',index=False,na_rep='NULL',columns='High')

In [ ]:
#csv文件的形式有很多，只需定义csv.Dialect的一个子类即可定义出新格式
import csv
class my_dialect(csv.Dialect):
    lineterminator='\n'
    delimiter=';'
    quotechar='"'
    quoting=csv.QUOTE_MINIMAL

path=r'data_test\ex7.txt'
f=open(path,'r')
reader = csv.reader(f, dialect=my_dialect)

### 6.1.6 JSON数据(JavaScript Object Notation)

In [ ]:
import json
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""
#json中的'null'就是python的'None'
#loads把json对象转化为Python对象
b=json.loads(obj)
#dumps把Python对象转换为JSON
a=['a','julia','Java','c++']
b=json.dumps(a)
#如何将JSON对象转化为DataFrame取决于自己的需要
import pandas as pd
df=pd.DataFrame(b['siblings'],columns=['age'])
#pandas.read_json可以将特别格式的JSON数据集转换为Serise或者DataFrame
df1=pd.read_json('D:\\文档\\Python Scripts\\data_test\\example.json')
#把DataFrame输出为json
print(df1.to_json())

## 6.2 二进制数据格式

### 6.2.1 实现数据的高效二进制格式存储最简单的办法之一是使用Python内置的pickle序列化

>Pickle序列化。pandas对象都有一个用于将数据以pickle格式保存到磁盘上的to_pickle方法：

In [1]:
import numpy as np
import pandas as pd

In [5]:
frame=pd.read_csv('D:\\文档\\Python Scripts\\data_test\\ex1.csv',index_col='message')
frame.head()
frame.to_pickle('D:\\文档\\Python Scripts\\data_test\\frame_pickle')
#读取
pd.read_pickle('D:\\文档\\Python Scripts\\data_test\\frame_pickle')

,0,a,b,c,d
message,,,,,
hello,1,1,2,3,4
world,2,5,6,7,8
foo,3,9,10,11,12


,0,a,b,c,d
message,,,,,
hello,1,1,2,3,4
world,2,5,6,7,8
foo,3,9,10,11,12


### 6.2.2 HDF5（hierarchical data format）

>每个HDF5文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据支持分块读写.

>如果需要本地处理海量数据，我建议你好好研究一下PyTables和h5py，看看它们能满足你的哪些需求。由于许多数据分析问题都是IO密集型（而不是CPU密集型），利用HDF5这样的工具能显著提升应用程序的效率。

In [ ]:
frame=pd.DataFrame({'a':np.random.randn(100)})
frame2=pd.DataFrame(np.random.randn(30,30))
store=pd.HDFStore('data_test\\mydata.h5')

store['obj1']=frame
store['obj2']=frame2
store
#HDF5文件中的对象可以通过与字典一样的API进行获取
store['obj1']
store['obj2']
#HDFStore支持两种存储模式，'fixed'和'table'，后者更慢，但是支持使用特殊语法进行查询操作
#put是store['obj2']=frame的显示版本,允许我们设置其它的选项，比如格式
store.put('obj2',frame,format='table')
store.select('obj2',where=['index >= 10 and index <= 15','columns=[3,4]'])
frame.to_hdf('data_test\\mydata.h5','obj3',format='table')
pd.read_hdf('data_test\\mydata.h5','obj3',where=['index < 5'])

### 6.2.3 Excel文件处理

>pandas的ExcelFile类或者pandas.read_excel函数支持读取存储Excel文件

In [ ]:
xlsx=pd.ExcelFile(r"D:\文档\Python Scripts\data_test\hw3data.xlsx")
a=pd.read_excel(xlsx,'日收益率计算')
#如果要读取一个文件中的多个表单，创建ExcelFile会更快
#但你也可以将文件名直接传递到pandas.read_excel
path=r"D:\文档\Python Scripts\data_test\hw3data.xlsx"
b=pd.read_excel(path)

**写入Excel文件**

In [6]:
import pandas_datareader.data as web
import pandas

code='600519.ss'
start='2019-4-22'
end='2019-4-26'
info=web.get_data_yahoo(code,start,end)
info.head()
writer=pd.ExcelWriter('D:\\文档\\Python Scripts\\data_test\info.to_excel')
info.to_excel(writer,'sheet1')
writer.save()